In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler

# 1. Cargamos los datos

Primero, vamos a importar los datos desde los archivos CSV que se encuentran en la carpeta historial pilas. Necesitamos unir un total de 186 archivos en un solo conjunto de datos.

In [11]:
# Ruta de la carpeta donde están los archivos CSV
folder_path = r"../data/raw/train/HistorialPilas"

# Obtener la lista de todos los archivos CSV en la carpeta
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Crear una lista para almacenar los DataFrames
dataframes = []

# Leer cada archivo CSV y agregarlo a la lista de DataFrames
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_pilas = pd.concat(dataframes, ignore_index=True)

# Mostrar una vista previa del DataFrame combinado
df_pilas.head()

# (Opcional) Mostrar los primeros registros para comprobar el resultado
# combined_df.head()

,Index,Time,Period,Seconds since,Mound,Comment,"""LSF"" [CurrentAnalysis.Dry basis]","""LSF"" [Rolling.Analysis1.Dry basis]","""SM"" [Rolling.Analysis1.Dry basis]","""IM"" [Rolling.Analysis1.Dry basis]",...,"""SiO2"" [Rolling.Analysis1.Dry basis]","""SM"" [CurrentAnalysis.Dry basis]","""CaO"" [CurrentAnalysis.Dry basis]","""MgO"" [CurrentAnalysis.Dry basis]","""IM"" [CurrentAnalysis.Dry basis]","""Fe2O3"" [CurrentAnalysis.Dry basis]","""Al2O3"" [CurrentAnalysis.Dry basis]","""SiO2"" [CurrentAnalysis.Dry basis]","""Tph"" [CurrentAnalysis.Dry basis]","""Tons"" [CurrentProduct.Dry basis]"
0,1,01/01/2023 14:11:30,60.060,0.00,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,85.36614,89.48588,2.90785,2.37234,...,14.88611,4.26911,41.25781,2.36092,1.65372,1.40812,2.32864,15.95262,1118.86,NaN
1,2,01/01/2023 14:12:30,60.058,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,87.28462,89.37545,2.96634,2.33612,...,14.96167,3.38848,42.71772,1.19249,2.11807,1.49516,3.16685,15.79714,1121.64,NaN
2,3,01/01/2023 14:13:30,60.061,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,88.86168,89.51137,3.01566,2.30483,...,14.97599,3.82237,41.63558,2.91829,1.90047,1.3807,2.62398,15.30737,1162.18,NaN
3,4,01/01/2023 14:14:30,60.060,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,81.11225,89.28863,3.08354,2.27769,...,15.00664,3.90974,39.55174,3.97634,1.88807,1.41361,2.669,15.96195,1205.73,NaN
4,5,01/01/2023 14:15:30,60.061,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,73.10878,88.315,3.09132,2.29887,...,15.11692,2.93623,38.92155,3.27556,2.47363,1.6571,4.09905,16.90138,1041.25,NaN


Usamos una expresión regular (regex) para extraer únicamente el número de pila de los datos y lo añadimos como una nueva columna llamada idPila.

In [12]:
# Aplicar regex para extraer el primer número después de la palabra "Pila"
df_pilas['idPila'] = df_pilas[' Mound'].str.extract(r'Pila (\d+)', expand=False)
df_pilas

,Index,Time,Period,Seconds since,Mound,Comment,"""LSF"" [CurrentAnalysis.Dry basis]","""LSF"" [Rolling.Analysis1.Dry basis]","""SM"" [Rolling.Analysis1.Dry basis]","""IM"" [Rolling.Analysis1.Dry basis]",...,"""SM"" [CurrentAnalysis.Dry basis]","""CaO"" [CurrentAnalysis.Dry basis]","""MgO"" [CurrentAnalysis.Dry basis]","""IM"" [CurrentAnalysis.Dry basis]","""Fe2O3"" [CurrentAnalysis.Dry basis]","""Al2O3"" [CurrentAnalysis.Dry basis]","""SiO2"" [CurrentAnalysis.Dry basis]","""Tph"" [CurrentAnalysis.Dry basis]","""Tons"" [CurrentProduct.Dry basis]",idPila
0,1,01/01/2023 14:11:30,60.060,0.00,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,85.36614,89.48588,2.90785,2.37234,...,4.26911,41.25781,2.36092,1.65372,1.40812,2.32864,15.95262,1118.86,NaN,0123
1,2,01/01/2023 14:12:30,60.058,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,87.28462,89.37545,2.96634,2.33612,...,3.38848,42.71772,1.19249,2.11807,1.49516,3.16685,15.79714,1121.64,NaN,0123
2,3,01/01/2023 14:13:30,60.061,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,88.86168,89.51137,3.01566,2.30483,...,3.82237,41.63558,2.91829,1.90047,1.3807,2.62398,15.30737,1162.18,NaN,0123
3,4,01/01/2023 14:14:30,60.060,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,81.11225,89.28863,3.08354,2.27769,...,3.90974,39.55174,3.97634,1.88807,1.41361,2.669,15.96195,1205.73,NaN,0123
4,5,01/01/2023 14:15:30,60.061,60.06,Pila 0123 Sur[Pila 10322 norte],AC mantencion 03-03-2016 .,73.10878,88.315,3.09132,2.29887,...,2.93623,38.92155,3.27556,2.47363,1.6571,4.09905,16.90138,1041.25,NaN,0123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142388,1481,02/01/2024 12:05:24,60.061,60.06,Pila 9723 Sur[Pila 9723 Sur],AC mantencion 03-03-2016 .,83.74801,83.60223,2.88241,2.1669,...,2.49274,39.04276,NaN,2.22224,1.80959,4.02134,14.53497,1117.94,28101.5464,9723
142389,1482,02/01/2024 12:06:24,60.059,60.06,Pila 9723 Sur[Pila 9723 Sur],AC mantencion 03-03-2016 .,88.95831,84.26865,2.85635,2.17833,...,2.86884,42.28739,NaN,1.87336,1.83244,3.43281,15.10514,1155.08,28120.7978,9723
142390,1483,02/01/2024 12:07:24,60.059,60.06,Pila 9723 Sur[Pila 9723 Sur],AC mantencion 03-03-2016 .,98.72054,85.62954,2.84576,2.18304,...,2.71152,42.38972,NaN,2.51971,1.415,3.56538,13.50436,1201.21,28140.8179,9723
142391,1484,02/01/2024 12:08:24,60.061,60.06,Pila 9723 Sur[Pila 9723 Sur],AC mantencion 03-03-2016 .,93.11037,86.90231,2.7919,2.23725,...,2.61297,42.05946,NaN,2.69241,1.46462,3.94335,14.13088,1139.05,28159.8021,9723


Ahora cargamos el archivo excel de los datos de calidad

In [13]:
df_QCX = pd.read_excel("../data/raw/train/dataQCX.xlsx")
df_QCX.head()

,fecha,fecha-hora,id_eq,Muestra #,FCAO,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,K2O,Na2O,C3S,C2S,C3A,C4AF,A/S,idPila
0,2019-01-01,2019-01-01 00:00:38,Aus_RM323,1478524,--,14.16677,3.409784,1.757292,42.01734,2.359858,0.104489,0.44943,0.199408,--,15.046283,6.06259,5.34744,6.123060,NaN
1,2019-01-01,2019-01-01 00:40:18,Man_CLK461,1478528,1.424847,21.01248,5.194796,2.515397,66.15322,3.540236,1.639114,0.640906,0.377616,65.318701,6.590723,9.510158,7.654353,0.630035,NaN
2,2019-01-01,2019-01-01 00:40:35,Aus_CLK463,1478529,1.463604,20.91105,5.145436,2.433739,66.25763,3.455422,1.649539,0.651268,0.351385,66.805214,5.059468,9.517519,7.405868,0.610880,NaN
3,2019-01-01,2019-01-01 00:44:52,Man_CLK462,1478530,1.707553,21.3169,5.215492,2.798341,67.94087,3.63804,0.128239,0.242341,0.25485,68.588091,4.128831,9.086261,8.515352,4.172559,NaN
4,2019-01-01,2019-01-01 01:00:38,Aus_RM323,1478531,--,13.55528,3.289323,1.687194,42.26035,2.369276,0.110288,0.43218,0.197379,--,8.354752,5.861974,5.134131,5.644265,NaN


In [14]:
%store df_QCX

Stored 'df_QCX' (DataFrame)


In [15]:
%store df_pilas

Stored 'df_pilas' (DataFrame)
